In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import joblib
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
path_rds_diagnoses = "/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/UKBB_diagnoses_Jun2025/processed/diagnoses_extracted"

In [ ]:
df = pd.read_table("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/omics/olink_data.txt")
df = df[df["ins_index"]==0]
codings = pd.read_csv('/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/codings/coding143.tsv', sep='\t')
codings[['protein_abbr','protein_name']] = codings['meaning'].str.split(';',expand=True).rename(columns={0:'protein_abbr',1:'protein_name'})
codings.rename(columns={"coding":"protein_id"},inplace=True)
df_merged = pd.merge(codings, df, on=["protein_id"])
df_merged.rename(columns={"result":"Expression levels"},inplace=True)
df_merged = df_merged[df_merged["ins_index"] == 0]
df_merged = df_merged[["eid","protein_abbr","Expression levels"]]


In [ ]:
withdrawn = pd.read_csv("../../Feature_selection_ligh/withdrawnparticipants.csv")
first_col_name = withdrawn.columns[0]
ids_withdrawn_list = [first_col_name] + withdrawn.iloc[:, 0].tolist()

In [ ]:
len(ids_withdrawn_list)

In [ ]:
df_merged = df_merged[~df_merged["eid"].isin(ids_withdrawn_list)]
proteomics_df =  df_merged.pivot_table(index=['eid'], columns='protein_abbr', values='Expression levels').reset_index()

In [ ]:
covariates = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/proteomics/Processed_all_covariates.csv").iloc[:,1:]

covariate_cols = ["Age","Sex","Ethnicity","BMI","Season","fasting_time","sample_age","smoking","alcohol","Season"]

In [ ]:
covariates[covariate_cols].isna().sum()

In [ ]:
covariates = covariates.dropna(subset=covariate_cols)

In [ ]:
proteomics_df = proteomics_df[proteomics_df["eid"].isin(covariates["eid"])]

In [ ]:
proteomics_ids = proteomics_df["eid"].unique().tolist()

In [ ]:
len(proteomics_ids)

In [ ]:
# Load each disease DataFrame
pd_df =  pd.read_csv(f"{path_rds_diagnoses}/Parkinson.csv").iloc[:, 1:]
ad_df = pd.read_csv(f"{path_rds_diagnoses}/AlzheimersDisease.csv").iloc[:, 1:]
vd_df = pd.read_csv(f"{path_rds_diagnoses}/VascularDementia.csv").iloc[:, 1:]
msa_df = pd.read_csv(f"{path_rds_diagnoses}/MSA.csv").iloc[:, 1:]
pnp_df = pd.read_csv(f"{path_rds_diagnoses}/PNP.csv").iloc[:, 1:]
ftd_df = pd.read_csv(f"{path_rds_diagnoses}/frontotemporaldementia.csv")
ht_df = pd.read_csv(f"{path_rds_diagnoses}/huntingtondisease.csv")
allcauseparkinsonism_df = pd.read_csv(f"{path_rds_diagnoses}/AllCauseParkinsonism.csv")
ms_df = pd.read_csv(f"{path_rds_diagnoses}/multiple_sclerosis.csv")
hc_df = pd.read_csv(f"{path_rds_diagnoses}/Not_neurological_noantiparkinsonismdrug.csv").iloc[:, 1:]

# Assign disease name and unify the date column
def prepare_df(df, disease_label, date_col="date_diagnosis"):
    df = df.copy()
    df["Disease"] = disease_label
    df["date_diagnosis"] = pd.to_datetime(df["date_diagnosis"], errors="coerce")
    return df[["eid", "date_diagnosis", "Disease"]]

# Prepare each disease df
diagnosis_dfs = [
    prepare_df(ad_df, "AD"),
    prepare_df(vd_df, "VD"),
    prepare_df(ftd_df, "FTD"),
    prepare_df(ht_df, "HD"),
    prepare_df(ms_df, "MS"),
    prepare_df(pd_df, "PD"),
    prepare_df(msa_df, "MSA"),
    prepare_df(pnp_df, "PNP"),
]

# Concatenate all diseases
all_diseases_df = pd.concat(diagnosis_dfs, ignore_index=True)
all_diseases_df = all_diseases_df.dropna()
# Sort with NaNs last so we prefer valid dates
all_diseases_df = all_diseases_df.sort_values("date_diagnosis")
all_diseases_df = all_diseases_df.sort_values("date_diagnosis")

# Add healthy controls, assigning None to date and Disease

hc_df["Disease"] = "HC"

In [ ]:
pd_original = pd_df.copy(deep=True)

# Visualisation before handling duplicates

In [ ]:
duplicates = all_diseases_df[all_diseases_df["eid"].duplicated(keep=False)]
diagnosis_counts = duplicates.groupby("eid").size().reset_index(name="num_diagnoses")
disease_counts_total = all_diseases_df["Disease"].value_counts()
diagnosis_counts_all = all_diseases_df.groupby("eid").size().reset_index(name="num_diagnoses")

In [ ]:
all_diseases_df

In [ ]:
# Plot
plt.figure(figsize=(12, 6))
ax= disease_counts_total.plot(kind="barh")
plt.gca().invert_yaxis()  # most common disease on top
plt.xlabel("Number of Diagnoses")
plt.ylabel("Disease")
plt.title("Number of participants with each diagnosis (before removing duplicates)")
# Add text labels at the end of bars
for i, (count, name) in enumerate(zip(disease_counts_total, disease_counts_total.index)):
    ax.text(count + max(disease_counts_total) * 0.01, i, f"{count:,}", va='center')

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=diagnosis_counts_all, x="num_diagnoses")
plt.title("Number of diagnoses per duplicated eid")
plt.xlabel("Number of Diagnoses")
plt.ylabel("Number of eids")

# Add numbers on top of bars
ax = plt.gca()
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2, height + 1, f'{int(height)}', 
            ha='center', va='bottom')

plt.show()


In [ ]:
disease_repeats = duplicates.groupby(["eid", "Disease"]).size().reset_index(name="count")
disease_counts_all = all_diseases_df.groupby(["eid", "Disease"]).size().reset_index(name="count")
pivot = disease_counts_all.pivot(index="eid", columns="Disease", values="count").fillna(0)


In [ ]:
# keep only eids with 3+ diagnoses
filtered_three = pivot[pivot.sum(axis=1) >= 3]
plt.figure(figsize=(12, 6))
sns.heatmap(filtered_three, cmap="Blues", linewidths=0.5, cbar=False)
plt.title("Eids with more than 2 diagnoses")
plt.xlabel("Disease")
plt.ylabel("eid")
plt.tight_layout()
plt.show()


In [ ]:
# keep only eids with 3+ diagnoses
filtered_four = pivot[pivot.sum(axis=1) >= 4]

plt.figure(figsize=(12, 6))
sns.heatmap(filtered_four, cmap="Blues", linewidths=0.5, cbar=False)
plt.title("Eids with more than 4 diagnoses")
plt.xlabel("Disease")
plt.ylabel("eid")
plt.tight_layout()
plt.show()


In [ ]:
binary_bool = pivot.astype(bool)


In [ ]:
from upsetplot import from_indicators

data_for_upset = from_indicators(binary_bool.columns, binary_bool)

In [ ]:
import matplotlib.pyplot as plt
from upsetplot import UpSet

plt.figure(figsize=(10, 6))
upset = UpSet(data_for_upset, subset_size='count', show_counts='%d', facecolor='navy')
upset.plot()

# Add label below the disease names
plt.xlabel("")

plt.title("Diagnosis combinations per participant")
plt.show()


# Removing PD with other diagnoses 

In [ ]:
pd_people = pd_df["eid"].unique().tolist()

# Find diagnoses of people that have received PD diagnosis
pd_only = all_diseases_df[all_diseases_df["eid"].isin(pd_people)]
pd_only = pd_only.drop_duplicates(subset=["Disease","eid"])
# Count number of diagnoses for each pd participant
eid_counts = pd_only["eid"].value_counts()

# Collect eids that occur more than once
removing_eids = eid_counts[eid_counts > 1].index

# Filter the dataframe to remove those so that we have "clean PD" and other disease groups without PD
filtered_df = all_diseases_df[~all_diseases_df["eid"].isin(removing_eids)]

In [ ]:
len(removing_eids)

In [ ]:
pd_only

In [ ]:
duplicates_filtered = filtered_df[filtered_df["eid"].duplicated(keep=False)]
diagnosis_counts_filtered = duplicates_filtered.groupby("eid").size().reset_index(name="num_diagnoses")
disease_counts_total_filtered = filtered_df["Disease"].value_counts()
diagnosis_counts_filtered = filtered_df.groupby("eid").size().reset_index(name="num_diagnoses")

In [ ]:
# Plot
plt.figure(figsize=(12, 6))
ax= disease_counts_total_filtered.plot(kind="barh")
plt.gca().invert_yaxis()  # most common disease on top
plt.xlabel("Number of Diagnoses")
plt.ylabel("Disease")
plt.title("Number of participants with each diagnosis (after removing duplicates)")
# Add text labels at the end of bars
for i, (count, name) in enumerate(zip(disease_counts_total, disease_counts_total.index)):
    ax.text(count + max(disease_counts_total) * 0.01, i, f"{count:,}", va='center')

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=diagnosis_counts_filtered, x="num_diagnoses")
plt.title("Number of diagnoses per duplicated eid after removing PD participants with duplicate diagnosis")
plt.xlabel("Number of Diagnoses")
plt.ylabel("Number of eids")

# Add numbers on top of bars
ax = plt.gca()
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2, height + 1, f'{int(height)}', 
            ha='center', va='bottom')

plt.show()


# Visualisation after handling PD patients with more than one neurodegenerative diagnosis

In [ ]:
# First, group and merge diseases
combined_diagnoses = (
    filtered_df
    .sort_values("date_diagnosis")  # make sure earliest dates come first
    .groupby("eid")
    .agg({
        "Disease": lambda x: " / ".join(sorted(x.unique())),
        "date_diagnosis": "first"  # earliest date after sorting

    })
    .reset_index()
)

# Mark rows containing multiple diagnoses
combined_diagnoses["multi_disease"] = combined_diagnoses["Disease"].str.contains("/")

# Sort by multi_disease descending so that '/' rows come first
combined_diagnoses = combined_diagnoses.sort_values(by="multi_disease", ascending=False)

# Drop duplicates keeping the first (i.e., multi-disease row if it exists)
final_combined = combined_diagnoses.drop_duplicates(subset=["eid"], keep="first").drop(columns="multi_disease")

print(final_combined)


In [ ]:
final_combined

In [ ]:
# Boolean mask of duplicated EIDs
duplicates_mask = final_combined["eid"].duplicated(keep=False)  # keep=False marks all duplicates as True

# Filter the dataframe to only show duplicated rows
duplicated_eids = final_combined[duplicates_mask]

# If you only want the list of duplicated EIDs (unique)
duplicated_eid_list = final_combined["eid"].unique()



In [ ]:
combined_diagnoses[combined_diagnoses["eid"].isin(duplicated_eid_list)]["Disease"]

In [ ]:
diagnosis_counts = final_combined.groupby("eid").size().reset_index(name="num_diagnoses")
disease_counts_final = final_combined["Disease"].value_counts()


In [ ]:
disease_counts_final

In [ ]:
# Plot
plt.figure(figsize=(12, 6))
ax= disease_counts_final.plot(kind="barh")
plt.gca().invert_yaxis()  # most common disease on top
plt.xlabel("Number of Diagnoses")
plt.ylabel("Disease")
plt.title("Number of participants with each diagnosis (after removing duplicates)")
# Add text labels at the end of bars
for i, (count, name) in enumerate(zip(disease_counts_final, disease_counts_final.index)):
    ax.text(count + max(disease_counts_final) * 0.01, i, f"{count:,}", va='center')

plt.tight_layout()
plt.show()

In [ ]:
combined_final_proteomics = combined_diagnoses[combined_diagnoses["eid"].isin(proteomics_ids)]

In [ ]:
combined_final_proteomics

In [ ]:
diagnosis_counts_proteomics = combined_final_proteomics.groupby("eid").size().reset_index(name="num_diagnoses")
disease_counts_final_proteomics = combined_final_proteomics["Disease"].value_counts()
# Plot
plt.figure(figsize=(12, 6))
ax= disease_counts_final_proteomics.plot(kind="barh")
plt.gca().invert_yaxis()  # most common disease on top
plt.xlabel("Number of Diagnoses")
plt.ylabel("Disease")
plt.title("Number of participants with proteomics with each diagnosis (after removing duplicates)")
# Add text labels at the end of bars
for i, (count, name) in enumerate(zip(disease_counts_final_proteomics, disease_counts_final_proteomics.index)):
    ax.text(count + max(disease_counts_final_proteomics) * 0.01, i, f"{count:,}", va='center')

plt.tight_layout()
plt.show()

In [ ]:
combined_final_proteomics

In [ ]:
pd_df= filtered_df[filtered_df["Disease"] == "PD"]

In [ ]:
pd_df = pd_df[pd_df["eid"].isin(proteomics_ids)]
pd_df = pd_df[pd_df["date_diagnosis"].notna()]


In [ ]:
other_neurodegenerative_df = final_combined[~final_combined["eid"].isin(pd_df["eid"].unique().tolist())]

In [ ]:
other_neurodegenerative_df

In [ ]:
other_neurodegenerative_df = other_neurodegenerative_df[other_neurodegenerative_df["eid"].isin(proteomics_ids)]
hc_df = hc_df[hc_df["eid"].isin(proteomics_ids)]
pd_df = pd_df[pd_df["eid"].isin(proteomics_ids)]


In [ ]:
other_neurodegenerative_df

In [ ]:
pd_df.to_csv("PD_filtered.csv")

In [ ]:
pd_df = pd_df.dropna(subset="date_diagnosis")

In [ ]:
pd_df

In [ ]:
visit_df = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/Visit_dates.csv").iloc[:,1:3]


In [ ]:
pd_all = pd.merge(pd_df , visit_df , on="eid")

In [ ]:
pd_all

In [ ]:
pd_all["visit_date"] = pd.to_datetime(pd_all["p53_i0"], errors='coerce')
pd_all["diff_years"] = (pd_all["date_diagnosis"] - pd_all["visit_date"]).dt.days / 365.25

baseline= pd_all[pd_all["diff_years"] <= 2]
prodromals = pd_all[pd_all["diff_years"] > 2]

In [ ]:
prodromals

In [ ]:
baseline

In [ ]:
visit_df.rename(columns={"p53_i0":"visit_date"},inplace=True)

In [ ]:
hc_df = hc_df.drop_duplicates()

In [ ]:
removing_eids

In [ ]:
toremove = set(removing_eids) | set(prodromals["eid"].unique())


In [ ]:
training = proteomics_df[~proteomics_df["eid"].isin(toremove)]

In [ ]:
import numpy as np

training["Diagnosis"] = np.where(
    training["eid"].isin(baseline["eid"]), 
    1,  # if in baseline
    0  # else
)



In [ ]:
testing = proteomics_df[proteomics_df["eid"].isin(prodromals["eid"].unique())]
comorbid = proteomics_df[proteomics_df["eid"].isin(removing_eids)]

In [ ]:
training.set_index("eid",inplace=True)
testing.set_index("eid",inplace=True)
comorbid.set_index("eid",inplace=True)

In [ ]:
training.to_csv("Training_baseline.csv")
testing.to_csv("Testing_baseline.csv")
comorbid.to_csv("Comorbid_onds.csv")

In [ ]:
training["Diagnosis"].value_counts()